In [ ]:
# Insert the directory
#import sys
#sys.path.insert(0,'/fs04/sq58/Hardnet/harmonic-dense-network')

In [1]:
cd '/fs04/sq58/Hardnet/harmonic-dense-network/'

/fs04/sq58/Hardnet/harmonic-dense-network


In [2]:
from monai.visualize import blend_images
from monai.transforms import Compose, Activations, AsDiscrete
from monai.losses import DiceCELoss
from src.utils.train_utils import HardUnetTrainer
from src.data.ISLES_dataset_2D import ISLESDataModule_2D
from src.data.covid_dataset import CovidDataModule
from src.models.mseg_hardnet import HarDMSEG
from src.models.FCHardnet import FCHardnet
import torch
import json
import pytorch_lightning as pl
import wandb
from pytorch_lightning.loggers.wandb import WandbLogger
import torch.nn as nn
from monai.losses import DiceLoss

In [3]:
pl.seed_everything(42, workers=True)
device = "cuda" if torch.cuda.is_available() else "cpu"
#wandb.init(project="awa")

[rank: 0] Seed set to 42


In [4]:
# datamodule = ISLESDataModule(data_properties=data, batch_size=1, device=device)
with open(fr'./src/data/ISLES_dataset.json', 'r') as file:
    data = json.load(file)
datamodule = ISLESDataModule_2D(batch_size=64, data_properties=data, num_workers=5)
# # Total image to read in. In this case, it's 10 (for both train and val). With split = 0.7, 7 wll go to train and 3 will go to val
datamodule.setup()

/scratch/sq58/mbui0018/conda/envs/hardnet/lib/python3.11/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.croppad.dictionary CropForegroundd.__init__:allow_smaller: Current default value of argument `allow_smaller=True` has been deprecated since version 1.2. It will be changed to `allow_smaller=False` in version 1.5.
  warn_deprecated(argname, msg, warning_category)


[0]


In [5]:
# #Loadin the data according to the upper parameters
train_loader = datamodule.train_dataloader()

for batch_idx, batch in enumerate(train_loader):
    print(batch["image"].shape)
    print(torch.unique(batch["label"]))
    break

torch.Size([64, 1, 192, 192])
tensor([0., 1.], dtype=torch.float64)


In [6]:
!nvidia-smi

Tue Feb 11 17:21:52 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       On  | 00000000:C2:00.0 Off |                    0 |
| N/A   29C    P8               9W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
unet = FCHardnet(n_classes=1, in_channels=1).to(device)
lr = 0.002
momentum = 0.9
decay = 0.0005
opt = torch.optim.AdamW
sched = torch.optim.lr_scheduler.ConstantLR
loss = DiceLoss

model = HardUnetTrainer(model=unet, roi_size_h=64, roi_size_w=64, lr=lr, optim=opt, sched=sched, decay=decay, momentum=momentum)

/scratch/sq58/mbui0018/conda/envs/hardnet/lib/python3.11/site-packages/pytorch_lightning/utilities/parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


In [ ]:
# initialize Lightning's trainer.
trainer = pl.Trainer(
    accelerator="gpu",
    devices=1,
    max_epochs=600,
    check_val_every_n_epoch=20,
)

# train
trainer.fit(model, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/scratch/sq58/mbui0018/conda/envs/hardnet/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


[0]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type      | Params | Mode 
-------------------------------------------
0 | net  | FCHardnet | 4.1 M  | train
1 | loss | DiceLoss  | 0      | train
-------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.472    Total estimated model params size (MB)
309       Modules in train mode
0         Modules in eval mode
SLURM auto-requeueing enabled. Setting signal handlers.


/scratch/sq58/mbui0018/conda/envs/hardnet/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (45) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2:  22%|██▏       | 10/45 [00:02<00:07,  4.41it/s, v_num=5, mean_train_dice=0.293, train_loss=0.821] 

In [ ]:
post_trans = Compose([Activations(sigmoid=True), AsDiscrete(threshold=0.5)])
train_loader = datamodule.train_dataloader()
val_loader = datamodule.val_dataloader()
count = 0
unet = unet.to(device)
for val_data in val_loader:
    if count == 11:
          break
    print("------------ INDEX: ", count, "----------------")
    val_images, val_labels = val_data["image"].float().to(device), val_data["label"].float().to(device)
    blend_img = blend_images(val_images[0],val_labels[0])
    outputs = unet(val_images)
    post_trans_output = [post_trans(i) for i in outputs]
    blend_out = blend_images(val_images[0],post_trans_output[0])
    blend_check = blend_images(val_labels[0],post_trans_output[0])

    plt.figure()
    f, axarr = plt.subplots(1,5)
    axarr[0].imshow((blend_img.cpu()).permute(1, 2, 0))
    axarr[0].set_title("1")
    axarr[0].set_axis_off()

    axarr[1].imshow((outputs[0].detach().cpu()).permute(1, 2, 0))
    axarr[1].set_title("2")
    axarr[1].set_axis_off()

    axarr[2].imshow((post_trans_output[0].detach().cpu()).permute(1, 2, 0))
    axarr[2].set_title("3")
    axarr[2].set_axis_off()

    axarr[3].imshow((blend_out.cpu()).permute(1, 2, 0))
    axarr[3].set_title("4")
    axarr[3].set_axis_off()

    axarr[4].imshow((blend_check.cpu()).permute(1, 2, 0))
    axarr[4].set_title("5")
    axarr[4].set_axis_off()
    plt.show()
    count+=1